In [1]:
import pandas as pd
from pathlib import Path
input_file = Path.cwd()/'receipts.csv'
df = pd.read_csv(input_file)

## Remove unwanted columns, add blank column and reorder

In [2]:
df.drop(['Given', 'Surname','Income','GST',], axis=1, inplace=True)

In [3]:
df["Cumulative"] = ""

In [4]:
df=df[['Number', 'Paid by', 'Date', 'Paid','Cumulative','Type', 'Payment_Type','Issued_By', 'Merchant Ref', 'Notes']]

In [5]:
df.head()

,Number,Paid by,Date,Paid,Cumulative,Type,Payment_Type,Issued_By,Merchant Ref,Notes
0,10659,Janis Hunter,2022-5-6 14:09:44,60.0,,Income,credit card,Denise Bird,CM469961317_Hunter,"Enrolment into ""Mahjong Friday Club - Chinese(..."
1,10660,Margita Chudziak,2022-5-9 10:37:32,80.0,,Income,EFT,Didi Mudigdo,NaN,"Enrolment into ""French Beginners Conversation(..."
2,10661,Jenny Shepherd,2022-5-9 14:37:55,-140.0,,Refund,credit card,Didi Mudigdo,CM857905570_Shepherd,"Enrolment Fee adjustment for ""Yoga Tuesday"", s..."
3,10662,Janine Murphy,2022-5-9 14:44:24,190.0,,Income,credit card,Didi Mudigdo,CM881356890_Murphy,"Enrolment into ""Botanical Watercolour Pencil a..."
4,10663,Sri Hart,2022-5-9 14:51:22,-10.0,,Refund,cash,Didi Mudigdo,NaN,"Enrolment Fee adjustment for ""The Inquisitive ..."


Add the column `Allocate` to contain MYOB allocation information

In [6]:
df['Allocate']=""

Populate Allocate with the course code (for now) TRYOUT NEW METHOD

In [7]:
#def allocate(notes,cost):
#    if notes.str.contains("Membership"):
#        if cost == 15.0:
#            return 'Term Membership'
#        else:
#            return 'Membership' 
#       endif
#    else:
#        return notes.str.extract(r'\((.*?)\)', expand=False)
#         return notes.str.extract(r'\((.*?)\)', expand=False)

In [8]:
#df['Allocate'] = df.apply(lambda x: allocate(df['Notes'], df['Paid']), axis=1)

In [9]:
df['Allocate'] = df['Notes'].str.extract(r'\((.*?)\)', expand=False).str.strip()

Check results

In [10]:
df.head()

,Number,Paid by,Date,Paid,Cumulative,Type,Payment_Type,Issued_By,Merchant Ref,Notes,Allocate
0,10659,Janis Hunter,2022-5-6 14:09:44,60.0,,Income,credit card,Denise Bird,CM469961317_Hunter,"Enrolment into ""Mahjong Friday Club - Chinese(...",CA9
1,10660,Margita Chudziak,2022-5-9 10:37:32,80.0,,Income,EFT,Didi Mudigdo,NaN,"Enrolment into ""French Beginners Conversation(...",L4
2,10661,Jenny Shepherd,2022-5-9 14:37:55,-140.0,,Refund,credit card,Didi Mudigdo,CM857905570_Shepherd,"Enrolment Fee adjustment for ""Yoga Tuesday"", s...",NaN
3,10662,Janine Murphy,2022-5-9 14:44:24,190.0,,Income,credit card,Didi Mudigdo,CM881356890_Murphy,"Enrolment into ""Botanical Watercolour Pencil a...",AC59
4,10663,Sri Hart,2022-5-9 14:51:22,-10.0,,Refund,cash,Didi Mudigdo,NaN,"Enrolment Fee adjustment for ""The Inquisitive ...",NaN


Now strip the digits (MAYBE NOT NEEDED)

In [11]:
# df['Allocate'] = df['Allocate'].replace('(\d+)','',regex=True)

In [12]:
df

,Number,Paid by,Date,Paid,Cumulative,Type,Payment_Type,Issued_By,Merchant Ref,Notes,Allocate
0,10659,Janis Hunter,2022-5-6 14:09:44,60.0,,Income,credit card,Denise Bird,CM469961317_Hunter,"Enrolment into ""Mahjong Friday Club - Chinese(...",CA9
1,10660,Margita Chudziak,2022-5-9 10:37:32,80.0,,Income,EFT,Didi Mudigdo,NaN,"Enrolment into ""French Beginners Conversation(...",L4
2,10661,Jenny Shepherd,2022-5-9 14:37:55,-140.0,,Refund,credit card,Didi Mudigdo,CM857905570_Shepherd,"Enrolment Fee adjustment for ""Yoga Tuesday"", s...",NaN
3,10662,Janine Murphy,2022-5-9 14:44:24,190.0,,Income,credit card,Didi Mudigdo,CM881356890_Murphy,"Enrolment into ""Botanical Watercolour Pencil a...",AC59
4,10663,Sri Hart,2022-5-9 14:51:22,-10.0,,Refund,cash,Didi Mudigdo,NaN,"Enrolment Fee adjustment for ""The Inquisitive ...",NaN
5,10664,Sri Hart,2022-5-9 14:52:07,-10.0,,Refund,cash,Didi Mudigdo,NaN,"Enrolment Fee adjustment for ""The Inquisitive ...",NaN
6,10665,Sri Hart,2022-5-9 14:53:00,-10.0,,Refund,cash,Didi Mudigdo,NaN,"Enrolment Fee adjustment for ""The Inquisitive ...",NaN
7,10666,Sri Hart,2022-5-9 14:53:30,-10.0,,Refund,cash,Didi Mudigdo,NaN,"Enrolment Fee adjustment for ""The Inquisitive ...",NaN
8,10667,Sri Hart,2022-5-9 14:54:27,-10.0,,Refund,cash,Didi Mudigdo,NaN,"Enrolment Fee adjustment for ""The Inquisitive ...",NaN


## Filter out non-credit card transactions
Now for the Boolean indexing. The rows we want to remove are 'Deposit'/'Saved' (from the Type column) and 'cash, 'EFT'/'cash' (from the Payment_Type) column. Constructing the Boolean index:

In [13]:
cash = df['Payment_Type']=='cash'
cheque = df['Payment_Type']=='cheque'
saved = df['Type']=='Saved'
eft = df['Payment_Type']=='EFT'
deposit = df['Type']=='Deposit'

And now apply and export:

In [14]:
df[~(cash|cheque|eft|saved|deposit)].to_csv('hurrah.csv',index=False)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Number        9 non-null      int64  
 1   Paid by       9 non-null      object 
 2   Date          9 non-null      object 
 3   Paid          9 non-null      float64
 4   Cumulative    9 non-null      object 
 5   Type          9 non-null      object 
 6   Payment_Type  9 non-null      object 
 7   Issued_By     9 non-null      object 
 8   Merchant Ref  3 non-null      object 
 9   Notes         9 non-null      object 
 10  Allocate      3 non-null      object 
dtypes: float64(1), int64(1), object(9)
memory usage: 920.0+ bytes


(Just changing one little thing to test Git-Cola)